<a href="https://colab.research.google.com/github/BernardoEstrada/eeg-profiling/blob/main/data_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Copy dataset from Drive
# Run this block if you have the zip dataset in your drive (EEGMMIDB)
#   Remember to change the drive path if the zip is not in the root
!mkdir -p /content/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0

!unzip -u /content/drive/MyDrive/eeg-motor-movementimagery-dataset-1.0.0 -d /content/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0

!mv /content/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/files/* /content/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/.
!rm -r /content/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/files

from IPython.display import clear_output
clear_output()

In [ ]:
# @title Copy transformed dataset from Drive
!mkdir transformed_data
!unzip -u /content/drive/MyDrive/tesina/transformed.zip -d /content/transformed_data

from IPython.display import clear_output
clear_output()

In [63]:
# @title Install dependencies
!pip install mne -q

In [5]:
# @title Import dependencies and def util functions

import os
import numpy as np
import matplotlib.pyplot as plt
import mne

MNE_PATH = "/content/mne_data"

def getAnnotationMapping(run: int):
  res = {
    "T0": "rest"
  }
  if run in [3, 4, 7, 8, 11, 12]:
    res["T1"] = "left_fist"
    res["T2"] = "right_fist"
  if run in [5, 6, 9, 10, 13, 14]:
    res["T1"] = "both_fists"
    res["T2"] = "both_feet"
  return res

tasks = {
  "na": "unknown_task",
  "T0A": "baseline_eo", # baseline, eyes open
  "T0B": "baseline_ec", # baseline, eyes closed
  "T1": "mv_one", # open and close corresponding fist
  "T2": "im_one", # imagine opening and closing the corresponding fist
  "T3": "mv_two", # open and close both fists or both feet
  "T4": "im_two", # imagine opening and closing both fists or both feet
}
def getTaskFromRun(run: int) -> str:
  if run == 1: return tasks["T0A"]
  if run == 2: return tasks["T0B"]
  try: return tasks[f"T{(run-2)%4}"]
  except: return tasks["na"]

def transformRunData(n_subject, n_run):
  # Get data and standarize
  f_name = mne.datasets.eegbci.load_data(n_subject, n_run, path=MNE_PATH)[0]
  raw = mne.io.read_raw_edf(f_name, preload=True).load_data()
  mne.datasets.eegbci.standardize(raw)

  # Rename and get events
  raw.annotations.rename(getAnnotationMapping(n_run))
  events_from_annot, event_dict = mne.events_from_annotations(raw)

  # Get event durations
  # Set second (end) value on annotations to first (start) value of next item -1
  for i in range(len(events_from_annot)):
    if i+1<len(events_from_annot):
      events_from_annot[i][1] = events_from_annot[i+1][0]-1
    else:
      events_from_annot[i, 1] = len(raw)

  # Get data in np arr
  time_data = raw.get_data().transpose()

  # Save all events
  for ev in event_dict:
    events = mne.event.pick_events(events_from_annot, include=[event_dict[ev]])
    data = np.array([time_data[e[0]:e[1]][:] for e in events], dtype=object)
    path = f'data/S{n_subject:03d}/R{n_run:02d}'
    os.makedirs(path, exist_ok=True)
    np.save(f'{path}/{ev}', data)

In [ ]:
# @title Transforms and zips ALL data. Takes a LONG time, if you already have the archive don't run this cell, use the 2nd cell instead
for s in range(109):
  for r in range(14):
    transformRunData(s+1, r+1)

!zip -r transformed.zip data

from IPython.display import clear_output
clear_output()

In [64]:
# !zip -r transformed.zip data
!mkdir /content/drive/MyDrive/tesina
!mv /content/drive/MyDrive/eeg-motor-movementimagery-dataset-1.0.0.zip /content/drive/MyDrive/tesina/.
!cp transformed.zip /content/drive/MyDrive/tesina/.

In [ ]:
n_subjects = 109
n_runs = 14
n_channels = 64
# min 37s
# max 181s
# mean 114s
n_time = 160 * 120

epochs = mne.Epochs(raw, events_from_annot, event_id=event_dict, preload=True)

In [18]:
from google.colab import output
output.enable_custom_widget_manager()
# %matplotlib widget

# print(ev, data.shape, len(max(data, key=len)), len(min(data, key=len)))